In [15]:

train_data_dir = "skin/train"
test_data_dir = "skin/test"

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Set GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Define data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)  # Normalize pixel values for test data

# Set the batch size
batch_size = 16  # Reduce batch size to save GPU memory

# Create training and testing data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)



Found 7452 images belonging to 6 classes.
Found 1644 images belonging to 6 classes.


In [17]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Pooling layer
    Dropout(0.5),
    layers.Dense(128, activation='relu'),  # Additional dense layer (adjust as needed)
    layers.Dense(6, activation='softmax')  # 6 classes (adjust as needed)
])

# Change the learning rate to a different value, e.g., 0.001
optimizer = Adam(learning_rate=0.001)

# Compile the model with the updated optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()  # View the model architecture

# Set the number of training epochs
epochs = 15


# Train the model with gradient checkpointing
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    run_eagerly=True  # Enable eager execution for gradient checkpointing
)

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_2   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 1280)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               163968    
                                                                 
 dense_5 (Dense)             (None, 6)                 774       
                                                                 
Total params: 2,422,726
Trainable params: 2,388,614
No

In [18]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)

# Print the test accuracy
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')



103/103 [==============================] - 8s 77ms/step - loss: 2.9672 - accuracy: 0.4410
Test Accuracy: 44.10%


In [19]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import decode_predictions, preprocess_input


# Load the test image
test_image_path = r'D:\skindisease\skin\test\Eczema Photos\eczema-acute-4.jpg'  # Replace with the path to your test image
img = image.load_img(test_image_path, target_size=(224, 224))  # Resize to match model input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Preprocess the image (normalize pixel values)
img_array = img_array / 255.0

# Make predictions using the trained model
predictions = model.predict(img_array)

# Interpret the predictions (assuming you have class labels)
class_labels = ['class1', 'class2', 'class3', 'class4', 'class5', 'class6']  # Replace with your class labels

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class label
predicted_class_label = class_labels[predicted_class_index]

# Print the predicted class label and confidence score
print(f'Predicted class: {predicted_class_label}')
print(f'Confidence score: {predictions[0][predicted_class_index] * 100:.2f}%')



1/1 [==============================] - 0s 67ms/step
Predicted class: class1
Confidence score: 70.86%


In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Set GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Define data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)  # Normalize pixel values for test data

# Set the batch size
batch_size = 16  # Reduce batch size to save GPU memory

# Create training and testing data generators
train_generator = train_datagen.flow_from_directory(
    "train",
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)

test_generator = test_datagen.flow_from_directory(
    "test",
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)

Found 16347 images belonging to 24 classes.
Found 4195 images belonging to 24 classes.


In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Set GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Define data generators for training and testing with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Set the batch size
batch_size = 32

# Create training and testing data generators
train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)

test_generator = test_datagen.flow_from_directory(
    'test',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer with dynamic class count
])

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Set the number of training epochs
epochs = 100  # Increase the number of epochs

# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

# Predict accuracy on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Found 16347 images belonging to 24 classes.
Found 4195 images belonging to 24 classes.
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 52, 52, 128)       73856     
                                 

In [21]:
import pickle

# Replace 'your_model' with your actual machine learning model object
your_model = history  # Your trained model

# Specify the file path where you want to save the PKL file
file_path = 'skin_disease.pkl'

# Save the model to the PKL file
with open(file_path, 'wb') as file:
    pickle.dump(your_model, file)

print(f'Model saved as {file_path}')


INFO:tensorflow:Assets written to: ram://62d7159f-f55c-4eee-b828-336f18a98902/assets


INFO:tensorflow:Assets written to: ram://62d7159f-f55c-4eee-b828-336f18a98902/assets


Model saved as skin_disease.pkl
